In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gastroesophageal_reflux_disease_(GERD)"
cohort = "GSE43580"

# Input paths
in_trait_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)"
in_cohort_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE43580"

# Output paths
out_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/GSE43580.csv"
out_gene_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE43580.csv"
out_clinical_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/GSE43580.csv"
json_path = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"SBV - Gene Expression Profiles of Lung Cancer Tumors - Adenocarcinomas and Squamous Cell Carcinomas"
!Series_summary	"This dataset encompassing the profiles of 150 lung cancer tumors was developed to serve as test dataset in the SBV IMPROVER Diagnostic Signature Challenge (sbvimprover.com). The aim of this subchallenge was to verify that it is possible to extract a robust diagnostic signature from gene expression data that can identify stages of different types of lung cancer. Participants were asked to develop and submit a classifier that can stratify lung cancer patients in one of four groups – Stage 1 of Adenocarcinoma (AC Stage 1), Stage 2 of Adenocarcinoma (AC Stage 2), Stage 1 of Squamous cell carcinoma (SCC Stage 1) or Stage 2 of Squamous cell carcinoma (SCC Stage 2). The classifier could be built by using any publicly available gene expression data with related histopathological information and was tested on the independent dataset describ

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression profiles of lung cancer tumors.
# It explicitly mentions gene expression data in "SBV - Gene Expression Profiles of Lung Cancer Tumors"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary for trait information (GERD)
# We need to check if GERD information is available

# Examining the rows for GERD (Gastroesophageal reflux disease) information
# Row 3 contains entries like "clinical diagnosis patient: Gastroesophageal reflux disease (Status Ongoing)"
trait_row = 3

# For age, row 1 contains "age at excision (years)" information
age_row = 1

# For gender, row 0 contains "gender: male" or "gender: female"
gender_row = 0

# Conversion functions for trait, age, and gender
def convert_trait(value):
    """Convert GERD trait status to binary (0 or 1)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if "Gastroesophageal reflux disease" is mentioned in the value
    if "Gastroesophageal reflux disease" in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous value"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # If value is NA or not a number, return None
    if value == "NA" or not value.replace('.', '', 1).isdigit():
        return None
    
    return float(value)

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Perform initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to a CSV file
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {'GSM1065725': [0.0, 65.0, 1.0], 'GSM1065726': [0.0, 61.0, 1.0], 'GSM1065727': [0.0, 43.0, 1.0], 'GSM1065728': [0.0, 44.0, 1.0], 'GSM1065729': [0.0, 60.0, 1.0], 'GSM1065730': [0.0, 58.0, 1.0], 'GSM1065731': [0.0, 67.0, 1.0], 'GSM1065732': [0.0, 52.0, 1.0], 'GSM1065733': [0.0, 43.0, 1.0], 'GSM1065734': [0.0, 66.0, 1.0], 'GSM1065735': [0.0, 47.0, 0.0], 'GSM1065736': [0.0, 56.0, 1.0], 'GSM1065737': [0.0, 62.0, 1.0], 'GSM1065738': [0.0, 69.0, 1.0], 'GSM1065739': [0.0, 60.0, 1.0], 'GSM1065740': [0.0, 47.0, 1.0], 'GSM1065741': [0.0, 69.0, 1.0], 'GSM1065742': [0.0, 49.0, 0.0], 'GSM1065743': [0.0, 68.0, 1.0], 'GSM1065744': [0.0, 65.0, 1.0], 'GSM1065745': [0.0, 52.0, 1.0], 'GSM1065746': [0.0, 67.0, 1.0], 'GSM1065747': [0.0, 70.0, 1.0], 'GSM1065748': [0.0, 60.0, 1.0], 'GSM1065749': [0.0, 46.0, 1.0], 'GSM1065750': [0.0, 52.0, 1.0], 'GSM1065751': [0.0, 66.0, 0.0], 'GSM1065752': [0.0, 63.0, 1.0], 'GSM1065753': [0.0, 57.0, 1.0], 'GSM1065754': [1.0, 56.0, 0.0], 'GSM10657

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 87
Header line: "ID_REF"	"GSM1065725"	"GSM1065726"	"GSM1065727"	"GSM1065728"	"GSM1065729"	"GSM1065730"	"GSM1065731"	"GSM1065732"	"GSM1065733"	"GSM1065734"	"GSM1065735"	"GSM1065736"	"GSM1065737"	"GSM1065738"	"GSM1065739"	"GSM1065740"	"GSM1065741"	"GSM1065742"	"GSM1065743"	"GSM1065744"	"GSM1065745"	"GSM1065746"	"GSM1065747"	"GSM1065748"	"GSM1065749"	"GSM1065750"	"GSM1065751"	"GSM1065752"	"GSM1065753"	"GSM1065754"	"GSM1065755"	"GSM1065756"	"GSM1065757"	"GSM1065758"	"GSM1065759"	"GSM1065760"	"GSM1065761"	"GSM1065762"	"GSM1065763"	"GSM1065764"	"GSM1065765"	"GSM1065766"	"GSM1065767"	"GSM1065768"	"GSM1065769"	"GSM1065770"	"GSM1065771"	"GSM1065772"	"GSM1065773"	"GSM1065774"	"GSM1065775"	"GSM1065776"	"GSM1065777"	"GSM1065778"	"GSM1065779"	"GSM1065780"	"GSM1065781"	"GSM1065782"	"GSM1065783"	"GSM1065784"	"GSM1065785"	"GSM1065786"	"GSM1065787"	"GSM1065788"	"GSM1065789"	"GSM1065790"	"GSM1065791"	"GSM1065792"	"GSM1065793"	"GSM1065794"	"GSM1065795"	"GSM1065796"	"GSM1065797"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# These identifiers like "1007_s_at", "1053_at", etc. are probe IDs from an Affymetrix microarray
# They are not human gene symbols, but rather probe identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE43580
Line 6: !Series_title = SBV - Gene Expression Profiles of Lung Cancer Tumors - Adenocarcinomas and Squamous Cell Carcinomas
Line 7: !Series_geo_accession = GSE43580
Line 8: !Series_status = Public on Oct 18 2013
Line 9: !Series_submission_date = Jan 17 2013
Line 10: !Series_last_update_date = Mar 25 2019
Line 11: !Series_pubmed_id = 23966112
Line 12: !Series_summary = This dataset encompassing the profiles of 150 lung cancer tumors was developed to serve as test dataset in the SBV IMPROVER Diagnostic Signature Challenge (sbvimprover.com). The aim of this subchallenge was to verify that it is possible to extract a robust diagnostic signature from gene expression data that can identify stages of diff


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# Extract the gene mapping from annotation data (probes to gene symbols)
# From the annotation dictionary, we can see:
# - 'ID' contains probe IDs like '1007_s_at' that match the gene expression data index
# - 'Gene Symbol' contains the gene symbols we want to map to

# 1. First get the gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 2. Apply the gene mapping to convert probe-level measurements to gene expression
# This will split values for probes that map to multiple genes and sum values for genes with multiple probes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 3. Look at the result
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")


Gene expression data shape after mapping: (21278, 150)
First 10 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# We already have gene_data from Step 6, so we don't need to read it from a file
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# Check clinical data structure
print("Clinical data columns:", clinical_df.columns.tolist())

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 3. Handle missing values
print("\nMissing values before handling:")
# The trait column is likely at position 0 based on the clinical_df structure
trait_col = 0  # First column in the linked data
print(f"  Trait missing: {linked_data[trait_col].isna().sum()} out of {len(linked_data)}")
if 1 in linked_data.columns:  # Age is likely at position 1
    print(f"  Age missing: {linked_data[1].isna().sum()} out of {len(linked_data)}")
if 2 in linked_data.columns:  # Gender is likely at position 2
    print(f"  Gender missing: {linked_data[2].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [0, 1, 2]]  # Non-clinical columns are genes
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

# Modify the handle_missing_values call to use the numeric column index instead of the trait name
cleaned_data = handle_missing_values(linked_data, trait_col)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    # Use the numeric column index for trait evaluation
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait_col)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 5. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Original gene data shape: (21278, 150)
Gene data shape after normalization: (19845, 150)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE43580.csv
Loaded clinical data shape: (3, 150)
Clinical data columns: ['GSM1065725', 'GSM1065726', 'GSM1065727', 'GSM1065728', 'GSM1065729', 'GSM1065730', 'GSM1065731', 'GSM1065732', 'GSM1065733', 'GSM1065734', 'GSM1065735', 'GSM1065736', 'GSM1065737', 'GSM1065738', 'GSM1065739', 'GSM1065740', 'GSM1065741', 'GSM1065742', 'GSM1065743', 'GSM1065744', 'GSM1065745', 'GSM1065746', 'GSM1065747', 'GSM1065748', 'GSM1065749', 'GSM1065750', 'GSM1065751', 'GSM1065752', 'GSM1065753', 'GSM1065754', 'GSM1065755', 'GSM1065756', 'GSM1065757', 'GSM1065758', 'GSM1065759', 'GSM1065760', 'GSM1065761', 'GSM1065762', 'GSM1065763', 'GSM1065764', 'GSM1065765', 'GSM1065766', 'GSM1065767', 'GSM1065768', 'GSM1065769', 'GSM1065770', 'GSM1065771', 'GSM1065772', 'GSM1065773', 'GSM1065774', 'GSM1065775', 'GSM1065776', 'GSM1065777', 'GSM1065778', 'GSM1065779', 'GSM1065780', 'GSM1065781', 'GSM1065782', 'GSM1065783

Data shape after handling missing values: (150, 19848)
For the feature '0', the least common label is '1.0' with 2 occurrences. This represents 1.33% of the dataset.
The distribution of the feature '0' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json
Data was determined to be unusable or empty and was not saved
